In [1]:
!sudo apt-get update 
!sudo apt-get install python3.8 

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2

#check python version
!python --version

! curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py 
! python3 get-pip.py --force-reinstall 

! pip install git+https://github.com/JustAnotherArchivist/snscrape.git

! pip install pandas 
! pip install datetime
! pip install pytz
! pip install tweepy
! pip install hazm

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [872 kB]
Get:13 http

     |████████████████████████████████| 60 kB 2.5 MB/s             
     |████████████████████████████████| 259 kB 8.5 MB/s            
     |████████████████████████████████| 66 kB 2.1 MB/s             
     |████████████████████████████████| 146 kB 9.0 MB/s            
     |████████████████████████████████| 316 kB 4.2 MB/s            
     |████████████████████████████████| 1.4 MB 28.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 233 kB 53.6 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=f8d6297753f32e18472798d388d5ff10763e7ca6ed2cf4c8777043c4c6125dfb
  Stored in directory: /root/.cache/pip/wheels/19/1d/3a/0a8c14c30132b4f9ffd796efbb6746f15b3d6bcfc1055a9346
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp38-cp38-linux_x86_64.whl size=154027 sha256=a0ac21d26c7919e41f949b054c8ce139d4f68b2c2483a75375ed7468c91fad2e
  Stored in director

In [40]:
prepared_code = """
import snscrape.modules.twitter as sntwitter
import pandas as pd
from random import random
from datetime import date
from multiprocessing.dummy import Pool
import time

# https://github.com/JustAnotherArchivist/snscrape/blob/master/snscrape/modules/twitter.py

class Twitter_scraper:
  
  def __init__(self,
               max_results: int,
               all_words = [],        # Example: ['what’s', 'happening'] · contains both “what’s” and “happening”
               exact_pharase=[],      # Example: ['happy hour'] · contains the exact phrase “happy hour”
               any_words = [],        # Example: ['cats', 'dogs'] · contains either “cats” or “dogs” (or both)
               none_words = [],       # Example: ['cats', 'dogs'] · does not contain “cats” and does not contain “dogs”
               hashtags = [],         # Example: ['#ThrowbackThursday'] or ['ThrowbackThursday'] · contains the hashtag #ThrowbackThursday
               mentioned_users = [],  # Example: ['@SFBART', '@Caltrain'] or ['SFBART', 'Caltrain'] · mentions @SFBART or mentions @Caltrain
               from_users = [],       # Example: ['@Twitter'] or ['Twitter'] · sent from @Twitter
               to_users = [],         # Example: ['@Twitter'] or ['Twitter'] · sent in reply to @Twitter
               with_links = True,
               with_replies = True,
               **kwargs):
    
    self.number_of_user = 0
    self.max_results = max_results
    self.all_words = Twitter_scraper.all_of_these_words(all_words)
    self.exact_pharase = f'\"{exact_pharase}\"' if exact_pharase else ''
    self.any_words = Twitter_scraper.any_of_these_words(any_words)
    self.none_words = Twitter_scraper.none_of_these_words(none_words)
    self.these_hashtags = Twitter_scraper.any_of_these_hashtags(hashtags)
    self.mentioned_users = Twitter_scraper.mentioning_these_users(mentioned_users)
    self.with_links = f'-filter:links' if not with_links else ''
    self.with_replies = f'-filter:replies' if not with_replies else ''

    self.query_dict = {'all_words':self.all_words, 'exact_pharase':self.exact_pharase,
                  'any_words':self.any_words, 'none_words':self.none_words, 'these_hashtags':self.these_hashtags,
                  'mentioned_users':self.mentioned_users, 'with_links': self.with_links, 'with_replies':self.with_replies}

    self.query_dict['from'] = Twitter_scraper.f_or_t_users(from_users, 'from')
    self.query_dict['to'] = Twitter_scraper.f_or_t_users(to_users, 'to')

    for key, value in kwargs.items():
        self.query_dict[key] = (f'({key}:{value})')


  @staticmethod
  def f_or_t_users(users, key):
    if not users:
      return ''
    tmp_list = [f'{key}:{user}' for user in users]
    return('(' + ' OR '.join(tmp_list) + ')')

  @staticmethod
  def all_of_these_words(all_words):
    if not all_words:
      return ''
    return ' '.join(all_words)

  @staticmethod
  def any_of_these_words(any_words):
    if not any_words:
      return ''
    return ('(' + ' OR '.join(any_words) + ')')

  @staticmethod
  def none_of_these_words(none_words):
    if not none_words:
      return ''    
    return ('-' + ' -'.join(none_words))

  @staticmethod
  def any_of_these_hashtags(hashtags):
    if not hashtags:
      return ''    
    tmp_list = ['#'+ h.replace('#','') for h in hashtags]
    return ('(' + ' OR '.join(tmp_list) + ')')

  @staticmethod
  def mentioning_these_users(users):
    if not users:
      return ''    
    tmp_list = ['@'+ h.replace('@','') for h in users]
    return ('(' + ' OR '.join(tmp_list) + ')')

  def create_query(self, query_dict):
    tmp_string = ''
    res = dict([(key, val) for key, val in 
           query_dict.items() if val])
    del query_dict
    query = ' '.join(res.values())
    del res

    return query


  def crawler(self, query, error_counter=0):
    # Creating list to append tweet data
    tweets_list = []
    try:
      # Using TwitterSearchScraper to scrape data and append tweets to list
      scraper = sntwitter.TwitterSearchScraper(query)
      i = 0
      for tweet in scraper.get_items(): #declare a username
          if i >= self.max_results: #check number and date
            break

          tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.replyCount, tweet.retweetCount,
                tweet.likeCount, tweet.user.username, tweet.lang, tweet.media, tweet.hashtags]) #declare the attributes to be returned
          i += 1
      
    except Exception as e:
      if 'Unable to find guest token' in str(e):
          error_counter += 1
          if error_counter > 3:
            error_counter = 0
            print("Sleep Time!")
            time.sleep(30.3 *60)
            print("Morning!")

          return self.crawler(query, error_counter)

      print(f"query: {query} , {e}")

    # Creating a dataframe from the tweets list above 
    tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Reply Count',
                                                    'Retweet Count', 'Like Count', 'Username', 'Lang', 'Media', 'Hashtags'])
    return tweets_df


  def basic_mode(self):
    query = self.create_query(self.query_dict)
    return self.crawler(query)

  def user_crawler(self, user):
    tmp_dict = self.query_dict.copy()
    tmp_dict['from'] = (f'(from:{user})')
    query = self.create_query(tmp_dict)
    del tmp_dict
    return self.crawler(query)


  def user_mode(self, user_list):   
    user_crawler = self.user_crawler 
    pool = Pool(22)
    df_list = pool.map(user_crawler, user_list)
    pool.close()
    pool.join()
    result_df = pd.concat(df_list, ignore_index=True)
    return result_df



topic = 'sports'
none_words=[] #['zodiac','bubble']
from_users = ['khabaronlinee', 'ilnanews', 'EtemadOnline', 'SharghDaily',
'hamshahrinews','PadDolat', 'IranNewspaper', 'isna_farsi', 'FarsNews_Agency',
'yjc___agency', 'Entekhab_News', 'IRNA_1313', 'Tasnimnews_Fa']
 
all_words=[]
any_words=['ورزش','فوتبال','لیگ','المپیک','والیبال','بسکتبال','مسابقات','تیم','فدراسیون','جام','مربی','بازیکنان','ورزشکاران']
until = '2022-01-27'
since = '2020-01-27'
scraper = Twitter_scraper(max_results=10**4, all_words=all_words,any_words=any_words,none_words=none_words,until=until, since=since, lang="fa", from_users=from_users, with_replies=False)
result = scraper.basic_mode()

result.to_csv('./drive/MyDrive/Colab Notebooks/topic_words/'+topic+'_tweets.csv', index=False)
print(result.shape)

# Example:
#all_words = ['ایران', 'است'] # contains both “ایران” and “است”
#lang="fa"
#scraper = Twitter_scraper(max_results=10**11, until="2020-01-01", since="2019-01-01", lang=lang, with_replies=False)
#result = scraper.basic_mode()

"""

with open("python.py", "w") as f:
  f.write(prepared_code)



In [41]:
! python python.py

(10000, 10)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


economics

any_words=['تجارت','مالی','اقتصادی','بورس','سکه','ارز','مالیات','قیمت','وام','اقتصاد']



---


health

any_words=['دارو','پاندمی','واگیر','مریضی','پزشکی','پزشکان','پزشک','بهداشت','درمان','بیماری','ویروس','واکسن','همه‌گیری','کرونا', 'بیمارستان', 'درمانگاه', 'بهداشتی', 'درمانی', 'تندرستی', 'دندانپزشک', 'فایزر','سینوفارم','آسترازنکا','برکت']

any_words=['دارو','پاندمی','واگیر','مریضی','پزشکی','پزشک','بهداشت','درمان','بیماری','واکسن','کرونا', 'بیمارستان', 'درمانگاه', 'بهداشتی', 'تندرستی', 'فایزر','سینوفارم']

any_words=['دارو','پاندمی','پزشکی','پزشک','بهداشت','درمان','بیماری','واکسن','کرونا', 'بهداشتی', 'تندرستی', 'فایزر','سینوفارم']


---


sports

any_words=['ورزش','فوتبال','لیگ','المپیک','والیبال','بسکتبال','مسابقات','تیم','فدراسیون','جام','مربی','بازیکنان','ورزشکاران']


---


art

any_words=['سینما','ادبیات','موسیقی','نقاشی','شعر','فیلم','بازیگر','هنرمند','هنر','هنرمندان']

---


tech

any_words=['تکنولوژی','فناوری','بیت کوین','بیت‌کوین','رمزارز','دیجیتال','اینترنت','شبکه','هک','رایانه','رایانه‌ای','بازی','موبایل']

---


transport

any_words=['جاده','اتوبان','بنزین','تاکسی','اتوبوس','مترو','هواپیما','هواپیمایی','ناوگان','قطار','پرواز','خودرو','ترافیک']

---


education

any_words=['مدرسه','معلم','معلمان','دبیر','دبیران','مدارس','دانشگاه','دانشگاه‌ها','دانش‌آموزان','کنکور','دانشجویان','دانشجو','اساتید']

---


religion

any_words=['اسلام','قرآن','نماز','مسلمان','مسلمانان','شیعه','سنی','شیعیان','مسیحیت','یهودیت','ائمه','پیامبر']

---


geo

any_words=['خشکسالی','آلودگی','سیل','زلزله','برف','باران','بارش','بارنگی','بی‌آبی','هواشناسی','رانش','آفتابی','گرما']

---

social

any_words=['','','','','','','','','','','','','']

---


life_style

any_words=['','','','','','','','','','','','','']
